# Исследование надежности заемщиков.
## Описание проекта:
### Заказчик — кредитный отдел банка. Необходммо разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
### Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Вопросы:
### 1. Есть ли зависимость между количеством детей и возвратом кредита в срок?

### Шаг 1. Обзор данных

In [182]:
import pandas as pd
import numpy as np

df=pd.read_csv('data (1).csv')
print(df.info())
display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Описание данных:
- **children** — количество детей в семье
- **days_employed** — общий трудовой стаж в днях  
- **dob_years** — возраст клиента в годах
- **education** — уровень образования клиента
- **education_id** — идентификатор уровня образования
- **family_status** — семейное положение
- **family_status_id** — идентификатор семейного положения
- **gender** — пол клиента
- **income_type** — тип занятости
- **debt** — имел ли задолженность по возврату кредитов
- **total_income** — ежемесячный доход
- **purpose** — цель получения кредита

### Проблеммы в данных
- В столбцах days_employed и total_income присутствуют пропуски.
- В столбцах days_employed и children встречаются аномальные значения (далее будет показано).
- В данных присутствуют явные и неявные дубликаты. 

### Оценим процент пропусков:

In [183]:
print('Процент пропущенных значений в столбце "days_employed": ',np.round(df['days_employed'].isna().sum()/len(df['days_employed'])*100,3),'%')
print('Процент пропущенных значений в столбце "total_income": ',np.round(df['total_income'].isna().sum()/len(df['total_income'])*100,3),'%')

Процент пропущенных значений в столбце "days_employed":  10.1 %
Процент пропущенных значений в столбце "total_income":  10.1 %


##### Процент пропущенных значений в обоих столбцах ~10.1%
### Возможные причины появления пропусков в данных:
* **Ошибки при сборе данных:** 
    * клиент отказался предоставлять конфиденциальную информацию о своем стаже и доходе.
* **Человеческая ошибка:**
    * сотрудник банка забыл внести данные, или клиент пропустил эти поля в анкете.
* **Особенности статуса клиента:**
    * безработные, студенты, пенсионеры. Для этих категорий понятия "трудовой стаж" (в контексте текущей работы) и "ежемесячный доход" могут быть неопределенными или некорректными
* **Неприменимость для некоторых профессий:**

    * Фрилансеры, самозанятые, владельцы бизнеса. Для них традиционное понятие "стаж в днях на текущем месте работы" может быть размытым или не имеющим смысла. 
* **Крайне высока вероятность, что пропуски в этих двух столбцах находятся в одних и тех же строках.**


### Шаг 2.1 Заполнение пропусков

#### Избавимся от пропусков в total_income заполнив их медианным значением стобца( медиана будет устойчива к выбрасам, что важно в контексте информации о доходах)

In [184]:
df.loc[df['total_income'].isna(),'total_income']=df['total_income'].median()
print(df['total_income'].isna().sum())

0


### Шаг 2.2 Проверка данных на аномалии и исправления.

In [185]:
print(df['days_employed'].describe())

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64


В данных days_employed есть отрицательные значения( отрицательное кол-во отработанных дней). Заменим отрицательные значения медианным для этого столбца, т.к. амена на медиану не сильно искажает общее распределение возраст (как например замена на случайное или среднее арифметическое)

In [186]:
(df.loc[df['days_employed']<0,'days_employed'])=np.nan
df['days_employed']=df['days_employed'].fillna(df['days_employed'].median())
print(df['days_employed'].describe())

count     21525.000000
mean     365179.857140
std        8430.550659
min      328728.720605
25%      365213.306266
50%      365213.306266
75%      365213.306266
max      401755.400475
Name: days_employed, dtype: float64


### Шаг 2.3. Изменение типов данных.
Обратим внимнеие что столбец days_employed имеет тип float, изменим его на int

In [187]:
df['days_employed']=df['days_employed'].astype('int')
df['children']=df['children'].astype('int')
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


None

### Шаг 2.4. Удаление дубликатов.
В данных присутствуют строки-дубликаты, удалим их. Также обработаем неявные дубликаты. Например, в столбце education есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к одному регистру.

Помимо этого столбец children содержет отрицательные значения кол-ва детей в семье, удалим эти значения.

In [188]:
display('Данные по типу образования: ',df['education'].unique())

df['education']=df['education'].str.lower()

display('Данные по типу образования после обработки: ',df['education'].unique())

display('Данные по семейному статусу: ',df['family_status'].unique())

display("Данные по полу: ",df['gender'].unique())

display("Данные по типу дохода: ",df['income_type'].unique())

#Кол-во детей
display(df['children'].unique())
df.loc[(df['children']<0)|(df['children']>15),'children']=np.nan
df=df.dropna()

display('Дубликаты перед обработкой: ',df[df.duplicated()].count())
df=df.drop_duplicates()
display('Итог обработки: ',df[df.duplicated()].count())


'Данные по типу образования: '

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

'Данные по типу образования после обработки: '

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

'Данные по семейному статусу: '

...

array([ 1,  0,  3,  2, -1,  4, 20,  5])

'Дубликаты перед обработкой: '

children            71
days_employed       71
dob_years           71
education           71
education_id        71
family_status       71
family_status_id    71
gender              71
income_type         71
debt                71
total_income        71
purpose             71
dtype: int64

'Итог обработки: '

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Перед поиском были проверены данные по стобцам имеющим категориальные признаки с помощью .unique(); Затем был изменен регистр всех значений столбца, чтобы стандартизировать данные.
Для поиска был использован df.duplicated(), т.к. он прост в использовании и наглядно показывает, какие именно строки являются повторениями. Это позволяет провести анализ перед тем, как безвозвратно удалить данные.
Для удаления был использован df.drop_duplicates()

Возможные причины появления дубликатов:
1. ошибки при слиянии данных
2. технические сбои
3. отсутствие ограничений по уникальности в базе данных:

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новых датафрейма, в которых:
- каждому уникальному значению из education соответствует уникальное значение education_id — в первом;
- каждому уникальному значению из family_status соответствует уникальное значение family_status_id — во втором.

Удалим из исходного датафрейма столбцы education и family_status, оставив только их идентификаторы: education_id и family_status_id. Новые датафреймы — это те самые «словари», к которым мы сможете обращаться по идентификатору.

In [189]:
education_dict=df.loc[:,'education':'education_id']
family_status_dict=df.loc[:,'family_status':'family_status_id']

df=df.drop(['education','family_status'],axis=1)
#display(df)

display(education_dict)
display(family_status_dict)

,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1
3,среднее,1
4,среднее,1
...,...,...
21520,среднее,1
21521,среднее,1
21522,среднее,1
21523,среднее,1


,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,1
...,...,...
21520,гражданский брак,1
21521,женат / замужем,0
21522,гражданский брак,1
21523,женат / замужем,0


### Шаг 2.6. Категоризация дохода.

Произведем категоризацию данных о доходе, по правилам описанным в функции income_category. В последствии будет проще проводить анализ зависимости уровня достатка и факта погашения кредита в срок.

In [190]:
def income_category(x):
    if x<=30000:
        return "E"
    elif x<=50000:
        return 'D'
    elif x<=200000:
        return 'C'
    elif x<=1000000:
        return 'B'
    else:
        return 'A'

df['total_income_category']=df['total_income'].apply(income_category)
display(df)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1.0,365213,42,0,0,F,сотрудник,0,253875.639453,покупка жилья,B
1,1.0,365213,36,1,0,F,сотрудник,0,112080.014102,приобретение автомобиля,C
2,0.0,365213,33,1,0,M,сотрудник,0,145885.952297,покупка жилья,C
3,3.0,365213,32,1,0,M,сотрудник,0,267628.550329,дополнительное образование,B
4,0.0,340266,53,1,1,F,пенсионер,0,158616.077870,сыграть свадьбу,C
...,...,...,...,...,...,...,...,...,...,...,...
21520,1.0,365213,43,1,1,F,компаньон,0,224791.862382,операции с жильем,B
21521,0.0,343937,67,1,0,F,пенсионер,0,155999.806512,сделка с автомобилем,C
21522,1.0,365213,38,1,1,M,сотрудник,1,89672.561153,недвижимость,C
21523,3.0,365213,38,1,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля,B


### Шаг 2.7. Категоризация целей кредита.
Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:
- 'операции с автомобилем',
- 'операции с недвижимостью',
- 'проведение свадьбы',
- 'получение образования'.

Например, если в столбце purpose находится подстрока 'на покупку автомобиля', то в столбце purpose_category должна появиться строка 'операции с автомобилем'.


In [191]:
#display(df['purpose'].unique())

def purpose(x):
    if 'образов' in x:
        return 'получение образования'
    if 'свадьб' in x:
        return 'проведение свадьбы'
    if 'автомоб' in x:
        return 'операции с автомобилем'
    if ('недвижим' in x) or ('жил' in x):
        return 'операции с недвижимостью'
    else:
        return 'другое'  
    
df['purpose_category']=df['purpose'].apply(purpose)
#display(df)
#display(df[df['purpose_category'].isna()])

## Ответы на вопросы.

### **Вопрос 1** "Влияние кол-ва детей на возрат кредита в срок":

In [192]:

pivot_table_1 = df.pivot_table(
    values=['debt', 'days_employed'],
    index='children',
    aggfunc={
        'debt': 'sum',           
        'days_employed': 'count'  
    }
)

pivot_table_1['Процент не имеющих долга']=np.round((pivot_table_1['days_employed']-pivot_table_1['debt'])/pivot_table_1['days_employed']*100,2)
pivot_table_1=pivot_table_1.rename(columns={'days_employed':'Всего семей','debt':'Семьи с задолжностями'})
display(pivot_table_1)


,Всего семей,Семьи с задолжностями,Процент не имеющих долга
children,,,
0.0,14091,1063,92.46
1.0,4808,444,90.77
2.0,2052,194,90.55
3.0,330,27,91.82
4.0,41,4,90.24
5.0,9,0,100.00


#### По резульатам анализа не наблюдается сильного или однозначного влияния количества детей на вероятность возврата кредита в срок. Процент добросовестных заемщиков остается стабильно высоким (около 90-92%) независимо от того, есть ли в семье дети и сколько их.

### **Вывод:** Наличие детей не является решающим фактором кредитной дисциплины.

### **Вопрос 2** "Есть ли зависимость уровня образования на возврат кредита в срок":

In [193]:
education_dict['debt']=df['debt']
#display(education_dick)
pivot_table_2=education_dict.pivot_table(
    values=['education_id','debt'],
    index='education',
    aggfunc={
        'education_id':'count',
        'debt':'sum'
    }
)
pivot_table_2=pivot_table_2.rename(columns={'debt':'Имеют долг','education_id':'Всего взяли кредит'})
pivot_table_2['Процент не имеющих долга']=(pivot_table_2['Всего взяли кредит']-pivot_table_2['Имеют долг'])/pivot_table_2['Всего взяли кредит']*100
display(pivot_table_2)

,Имеют долг,Всего взяли кредит,Процент не имеющих долга
education,,,
высшее,278,5227,94.681462
начальное,31,282,89.007092
неоконченное высшее,68,741,90.823212
среднее,1355,15075,91.011609
ученая степень,0,6,100.000000


#### По резульатам анализа наблюдается четкий градиент — с ростом уровня образования (за исключением малых групп) растет и финансовая дисциплина: Начальное < Неоконченное высшее < Среднее < Высшее < Ученая степень. 
#### Детальная интерпретация по группам:
1. Высшее образование (Лидер надежности):
    * Процент возврата: 94.68%
    * Это наивысший показатель среди всех значимых групп.
    * Интерпретация: Данная группа является наиболее финансово дисциплинированной и надежной. Это может быть связано с более стабильным доходом, пониманием долгосрочных последствий просрочек и более ответственным финансовым планированием.

2. Ученая степень (Статистическая аномалия):
    * Процент возврата: 100%

    * **Важно!**: Размер выборки (6 человек) слишком мал для каких-либо выводов. Результат 100% является статистической аномалией. На практике нельзя ожидать, что все обладатели ученых степеней всегда будут возвращать кредиты.

3. Среднее образование (Средний показатель)
    * Процент возврата: 91.01%
    * Это самая многочисленная группа, и ее показатель близок к общему среднему по выборке. Данная группа представляет "базовый" уровень надежности.

4. Неоконченное высшее (Немного ниже среднего)
    * Процент возврата: 90.82%
    * Показатель немного хуже, чем у группы со средним образованием. Это может быть связано с нестабильным финансовым положением (студенты, недавние выпускники), которые еще не вышли на стабильный уровень дохода.

5. Начальное образование (Наибольший риск)
    * Процент возврата: 89.01%
    * Это наименьший показатель среди всех значимых групп.
    * Интерпретация: Данная группа демонстрирует наибольший кредитный риск. Это может коррелировать с более низким уровнем доходов, нестабильной занятостью или меньшей осведомленностью в вопросах управления финансами.
### **Вывод**:  Уровень образования показал себя как статистически значимый показатель, позволяющий дифференцировать заемщиков по уровню кредитного риска. Клиенты с высшим образованием продемонстрировали наивысшую надежность, в то время как клиенты с начальным образованием — наименьшую.

### **Вопрос 3** "Есть ли зависимость между семейным положением и возвратом кредита в срок?":

In [194]:
family_status_dict['debt']=(df.loc[df['family_status_id']==family_status_dick['family_status_id'],'debt'])
#display(family_status_dick)

pivot_table_3=family_status_dict.pivot_table(
    values=['family_status_id','debt'],
    index='family_status',
    aggfunc={
        'family_status_id':'count',
        'debt':'sum'
    }
)

#display(pivot_table_3['debt'].sum())
pivot_table_3['Процент не имеющих долга']=(pivot_table_3['family_status_id']-pivot_table_3['debt'])/pivot_table_3['family_status_id']*100
pivot_table_3=pivot_table_3.rename(columns={'debt':'Имеют долг','family_status_id':'Всего взяли кредит'})
display(pivot_table_3)



,Имеют долг,Всего взяли кредит,Процент не имеющих долга
family_status,,,
Не женат / не замужем,273,2796,90.236052
в разводе,84,1189,92.935240
вдовец / вдова,63,951,93.375394
гражданский брак,385,4134,90.686986
женат / замужем,927,12261,92.439442


#### По резульатам анализа наблюдается четкая зависимость между семейным положением и кредитной дисциплиной. Люди, состоящие в официальных отношениях (женаты/замужем, вдовцы, в разводе), демонстрируют более высокую финансовую надежность, чем одинокие и состоящие в гражданском браке.

#### Гражданский брак (90.69%) vs Официальный брак (92.44%)
- ##### Разница почти в 2 процентных пункта говорит о том, что официально зарегистрированные отношения связаны с большей финансовой ответственностью.
### **Вывод:** Семейные люди действительно показывают лучшую кредитную дисциплину

### **Вопрос 4 "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?":

In [195]:
pivot_table_4=df.pivot_table(
    values=['family_status_id','debt'],
    index='total_income_category',
    aggfunc={
        'family_status_id':'count',
        'debt':'sum'
    }
)
#display(pivot_table_4)
pivot_table_4['Процент не имеющих долга']=np.round((pivot_table_4['family_status_id']-pivot_table_4['debt'])/pivot_table_4['family_status_id']*100,2)
pivot_table_4=pivot_table_4.rename(columns={'debt':'Имеют долг','family_status_id':'Всего взяли кредит'})
display(pivot_table_4)


,Имеют долг,Всего взяли кредит,Процент не имеющих долга
total_income_category,,,
A,2,25,92.00
B,354,5013,92.94
C,1353,15922,91.50
D,21,349,93.98
E,2,22,90.91


#### На основе предоставленных данных не наблюдается статистически значимой зависимости между категорией дохода и вероятностью возврата кредита. Все категории демонстрируют очень стабильный и высокий уровень возврата (в диапазоне 91-94%). Небольшие различия, скорее всего, связаны с погрешностью, особенно в малых группах.
#### Анализ по категориям (с учетом размера выборки)
- Категория D: Показывает наивысший процент возврата (93.98%), но выборка недостаточно велика (349 заемщиков), чтобы считать это устойчивой закономерностью.

 - Категория B: Демонстрирует высокий и стабильный результат (92.94%) на солидной выборке (5013 заемщиков). Это надежный показатель.

- Категория C: Самая крупная группа (15922 заемщика) показывает результат 91.50%, что очень близко к общему среднему.

- Категории A и E: Имеют экстремальные значения (92.00% и 90.91%), но на очень малых выборках (25 и 22 заемщика соответственно). Эти данные статистически незначимы - изменение на 1-2 случая dramatically меняет проценты.

#### **Критически** важное замечание о статистической значимости:
- Размеры групп A, D и E (25, 349 и 22 человека) слишком малы для надежных выводов:
    - В группе E всего 2 дефолта - появление еще одного просрочившего повысит долг дефолта до 13.6%
    - В группе A аналогичная ситуация

### **Вывод**: Категория общего дохода не является статистически значимым предиктором кредитного дефолта в данном наборе данных.

### **Вопрос №5** "Как разные цели кредита влияют на его возврат в срок?":

In [196]:
pivot_table_5=df.pivot_table(
    values=['family_status_id','debt'],
    index='purpose_category',
    aggfunc={
        'family_status_id':'count',
        'debt':'sum'
    }
)

pivot_table_5['Процент не имеющих долга']=np.round((pivot_table_5['family_status_id']-pivot_table_5['debt'])/pivot_table_5['family_status_id']*100,2)
pivot_table_5=pivot_table_5.rename(columns={'debt':'Имеют долг','family_status_id':'Всего взяли кредит'})
display(pivot_table_5)

,Имеют долг,Всего взяли кредит,Процент не имеющих долга
purpose_category,,,
операции с автомобилем,400,4279,90.65
операции с недвижимостью,780,10751,92.74
получение образования,369,3988,90.75
проведение свадьбы,183,2313,92.09


#### По резульатам анализа наблюдается статистически значимая разница в кредитной дисциплине в зависимости от цели кредита. Кредиты на недвижимость и свадьбу показывают наивысшую возвратность, в то время как кредиты на автомобили и образование связаны с несколько более высоким риском.
#### Детальная интерпретация по целям кредита
1. Наиболее надежные цели (≥92%)
    * Операции с недвижимостью: 92.74% - НАИВЫСШАЯ надежность
    * Проведение свадьбы: 92.09% - высокая надежность
    * Объяснение:
        * Недвижимость - долгосрочные инвестиции, серьезное финансовое планирование, залоговая дисциплина
        * Свадьба - важное жизненное событие, часто связанное с поддержкой семьи, повышенной ответственностью

2. Менее надежные цели (∼90.7%)
    * Получение образования: 90.75%
    * Операции с автомобилем: 90.65% - НАИМЕНЬШАЯ надежность
    * Объяснение:
        * Образование - студенты без стабильного дохода, отложенная окупаемость инвестиций
        * Автомобиль - быстро обесценивающийся актив, дополнительные расходы на содержание
### **Вывод:** Цель кредита является статистически значимым предиктором кредитного поведения. Наивысшую надежность демонстрируют заемщики, берущие кредиты на недвижимость и свадьбу, тогда как кредиты на автомобили и образование связаны с повышенным риском дефолта.

## Общий вывод:

### На основе анализа данных выявлены значимые и незначимые факторы, влияющие на возврат кредита в срок. Вот сводная картина:
#### Статистически значимые факторы (оказывают влияние):
1. Образование (Вопрос 2)
    * Сильная зависимость: клиенты с высшим образованием демонстрируют наивысшую возвратность (94.68%), тогда как с начальным образованием — наименьшую (89.01%).
    * Рекомендация: учитывать уровень образования в скоринговых моделях.

2. Семейное положение (Вопрос 3)
    * Четкая зависимость: вдовцы/вдовы (93.38%), разведенные (92.94%) и официально женатые/замужние (92.44%) показывают лучшие результаты, чем одинокие (90.24%) и состоящие в гражданском браке (90.69%).
    * Рекомендация: формальный статус отношений связан с повышенной финансовой ответственностью.

3. Цель кредита (Вопрос 5)
    * Заметная разница: кредиты на недвижимость (92.74%) и свадьбу (92.09%) возвращаются лучше, чем на автомобили (90.65%) и образование (90.75%).
    * Рекомендация: дифференцировать кредитные условия в зависимости от цели займа.

#### Статистически незначимые факторы (не оказывают существенного влияния):
1. Количество детей (Вопрос 1)
    * Стабильность: процент возврата колеблется в узком диапазоне 90.5–92.5% независимо от количества детей.
    * Вывод: наличие детей не является определяющим фактором кредитной дисциплины.

2. Уровень дохода (Вопрос 4)
    * Единообразие: все доходные группы показывают схожие результаты (91–94% возврата).
    * Вывод: категория дохода не позволяет эффективно сегментировать заемщиков по риску.

### **Итог**: Надежность заемщика определяется скорее его социально-демографическими характеристиками (образование, семейный статус) и целевым назначением кредита, чем материальными факторами (доход, количество иждивенцев) в рамках данного исследования.